In [24]:

import regex as re
import pandas as pd
import numpy as np
from termcolor import colored
from urllib.request import urlopen
import json
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [25]:
%pwd 
%cd c:\\Users\\Lenovo\\Flask apps\\House-price-prediction\\

c:\Users\Lenovo\Flask apps\House-price-prediction


In [26]:
MAIN_DF_PATH = r'Dataset\calgary_cleaned_rentals_dataset.csv'
df = pd.read_csv(MAIN_DF_PATH)

In [27]:
city = 'calgary'
df.dropna(subset=['price', 'sq_feet'], inplace=True)
df.reset_index(inplace=True, drop=True)

assert len(df.rented.unique()) ==1
df.drop(columns=['rented'], inplace=True)
df.loc[:, 'city'].fillna(city, inplace=True)


In [28]:
# handling second rentals --ADD
def handle_second_rentals(df, info_cols = ['price', 'sq_feet', 'beds', 'baths']):
    """
    separates residence options that has two listings
    creates the second dataset
    then renames both dataset's cols to have info_cols names
    """
    info_cols_2 = [col + '2' for col in info_cols]

    # check for where at least two second house columns are not null
    df_second = df[(~df[info_cols_2[0]].isna())&(~df[info_cols_2[1]].isna())]

    df_second = df_second.drop(columns=info_cols)
    for col in info_cols:
        df_second = df_second.rename(columns={col + '2': col})

    df = df.drop(columns=info_cols_2)
    return df, df_second

df, df_second = handle_second_rentals(df, info_cols = ['price', 'sq_feet', 'beds', 'baths'])
df_total = pd.concat([df, df_second], axis=0)
df_total.reset_index(inplace=True, drop=True)

In [29]:
len(df_total)

2213

In [30]:
# --ADD
def clean_undefined_dates(df, null_substitute = 'call for availability'):
    """
    call for availability is represented as 3000-01-01 in the dataset
    so we replace these dates with a string
    """
    for i in range(len(df)):
        try: 
            df.loc[i, 'a'] = pd.to_datetime(df.loc[i, 'a'], format='%Y-%m-%d')
        except: 
            df.loc[i, 'a'] = null_substitute
            df.loc[i, 'availability'] = null_substitute
    df.reset_index(inplace=True, drop=True)
    return df


df_total = clean_undefined_dates(df_total)

In [31]:
# -- ADD
def create_month_col(df):
    for i in range(len(df)):
        try:
            df.loc[i, 'available_month'] = int(df.loc[i, 'a'].month)
        except:
            # if the value is 3001 meaning that it might be available
            df.loc[i, 'available_month'] = df.loc[i, 'a']
            
    return df
df_total = create_month_col(df_total)

In [33]:
for i in range(len(df_total)):
    try: int(df_total.loc[i, 'available_month'])
    except: print(df_total.loc[i, 'available_month'])

call for availability
call for availability
call for availability
call for availability
call for availability
call for availability
call for availability
call for availability
call for availability
call for availability
call for availability
call for availability
call for availability
call for availability
call for availability
call for availability
call for availability
call for availability
call for availability
call for availability
call for availability
call for availability
call for availability
call for availability
call for availability
call for availability
call for availability
call for availability
call for availability
call for availability
call for availability
call for availability
call for availability
call for availability
call for availability
call for availability
call for availability
call for availability
call for availability
call for availability
call for availability
call for availability
call for availability
call for availability
call for availability
call for a

In [18]:
# --ADD
def clean_baths_col(df):
    for i in range(len(df)):
        try: df.loc[i, 'baths'] = np.floor(float(df.loc[i, 'baths']))
        except: 
            df.loc[i, 'baths'] = 0
    return df

df_total = clean_baths_col(df_total)

In [19]:
# --ADD
df_total.loc[:, 'cats'] = df_total.loc[:, 'cats'].apply(lambda x : int(x))
df_total.loc[:, 'dogs'] = df_total.loc[:, 'dogs'].apply(lambda x : int(x))

In [20]:
# -- ADD

def clean_beds_col(df):

    for i in range(len(df)):
        val = df.loc[i, 'beds']
        try: df.loc[i, 'beds'] = int(val)
        except: 
            if re.findall('\d', val): 
                df.loc[i, 'beds'] = int(re.findall('\d', val)[0])
            else:
                df.loc[i, 'beds'] = 0
    return df

df_total = clean_beds_col(df_total)


In [21]:
# --ADD

def clean_sq_feet_col(df, remove_longer_than=50):

    unwanted = []
    for i in range(len(df)):
        record = df.loc[i, 'sq_feet']

        try:
            df.loc[i, 'sq_feet'] = int(float(record))
            

        except:
            # lengthy descriptions
            if len(record)>remove_longer_than or not re.findall('\d+', record):
                unwanted.append(i)
                print('not int found: ', record)

            # it is 700-800 balcony
            elif re.findall('[\w\.\!\- ]*\d+ *- *\d+[\w\.\!\- ]*', record):
                num1 = re.findall('\d+', record)[0]
                num2 = re.findall('\d+', record)[1]

                df.loc[i, 'sq_feet'] = int((int(num1) + int(num2))/2)

            # it is 700 + 500 balcony
            elif re.findall('[\w\.\!\- ]*\d+ *\+ *\d+[\w\.\!\- ]*', record):
                num1 = re.findall('\d+', record)[0]
                num2 = re.findall('\d+', record)[1]

                df.loc[i, 'sq_feet'] = int(num1) + int(num2)

            # ~600
            elif len(re.findall('\d+', record))==1:
                df.loc[i, 'sq_feet'] = int(re.findall('\d+', record)[0])

            # 2,000
            elif re.findall('\d+,*\d+', record):
                df.loc[i, 'sq_feet'] = int(re.findall('\d+,*\d+', '2,000')[0].replace(',', ''))

            else:
                print(df.loc[i, 'sq_feet']) 
                unwanted.append(i)
                
    df.drop(unwanted, inplace=True)
    df = df[df.sq_feet!=0]
    df.reset_index(inplace=True, drop=True)

    return df
    
df_total = clean_sq_feet_col(df_total)


not int found:  Large one bedroom
not int found:  ONE MONTH FREE ON A ONE YEAR LEASE  RENT IS $912/MONTH WITH INCENTIVE SPREAD OVER ONE YEAR
not int found:  varies
not int found:  ONE MONTH FREE ON A ONE YEAR LEASE  RENT IS $1004/MONTH WITH INCENTIVE SPREAD OVER ONE YEAR
not int found:  HUGE HOUSE
not int found:  1089 sq.ft (924 sq.ft interior + 165 sq.ft exterior)
not int found:  1550-1750 sq.ft. ***This will vary due to range of different units


In [22]:
import plotly.express as px
fig = px.scatter(df_total, x="sq_feet", y="price", color="type", trendline="ols")
fig.show()

In [23]:
df_total.to_csv('Dataset/calgary_cleaned_rentals_dataset_2.csv', index=False)

### Stats model exp

In [24]:
for i in range(len(df_total)):
    try: df_total.sq_feet[i] = int(df_total.sq_feet[i])#.dtype
    except: print( df_total.sq_feet[i])

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_6472\2767917806.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [9]:
import statsmodels.api as sm

model_lin = sm.OLS.from_formula("price ~ sq_feet", data=df_total)
result_lin = model_lin.fit()
#result_lin.summary()


NameError: name 'df_total' is not defined

In [204]:
import plotly.express as px
fig = px.scatter(df_total, x="sq_feet", y="price", color="community")
fig.show()

In [211]:
print('number of communities = ', len(df_total.community.unique()))

number of communities =  203


In [212]:
df_exp = df_total.groupby(['community']).count()[['city']].sort_values(by='city').reset_index()
fig = px.bar(df_exp, x='community', y='city')
fig.show()

In [ ]:
df_p = df_total

In [230]:
df_exp

,community,city
0,Abbeydale,1
1,Country Hills,1
2,Deer Ridge,1
3,Douglas Glen,1
4,Hamptons,1
...,...,...
198,Bankview,49
199,Victoria Park,50
200,Lower Mount Royal,58
201,Downtown,98


In [242]:
communities_with_at_least_n_records = 20
df_exp2 = pd.merge(left=df_exp, right=df_total[['community', 'price']], on='community')

df_exp3 = df_exp2[df_exp2['city']>communities_with_at_least_n_records]
fig = px.box(df_exp3, x="community", y="price",
             title="Box plot of popular comminities prices (right-most = most popular)")
fig.show()

In [202]:
df_total.head(1)

,availability,a,city,community,latitude,longitude,link,type,price,beds,sq_feet,baths,cats,dogs,utility_heat,utility_electricity,utility_water,utility_cable,utility_internet,available_month
0,Immediate,2000-01-01 00:00:00,Calgary,Cliff Bungalow,51.036696,-114.074492,/ab/calgary/rentals/apartment/studio/cliff-bungalow/357976,Apartment,525.0,studio,420,1,0,0,1,1,1,0,0,1.0


In [262]:
column_to_plot = 'baths'
df_exp = df_total.groupby([column_to_plot]).agg({'city': 'count', 'price':'mean'}).reset_index()
fig = px.box(df_total, x=column_to_plot, y='price')
fig.show()

In [263]:
df_total.dtypes

availability            object
a                       object
city                    object
community               object
latitude               float64
longitude              float64
link                    object
type                    object
price                  float64
beds                    object
sq_feet                 object
baths                   object
cats                     int64
dogs                     int64
utility_heat             int64
utility_electricity      int64
utility_water            int64
utility_cable            int64
utility_internet         int64
available_month         object
dtype: object

### Map

In [1]:
# Import necessary packages
import os 
import folium
from folium import plugins
import rioxarray as rxr
import earthpy as et
import earthpy.spatial as es

# Import data from EarthPy
data = et.data.get_data('colorado-flood')


# Set working directory to earth-analytics
os.chdir(os.path.join(et.io.HOME, 'earth-analytics', 'data'))

ModuleNotFoundError: No module named 'rioxarray'

In [4]:
!pip install rioxarray


  Using cached rioxarray-0.10.3.tar.gz (47 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached xarray-2022.3.0-py3-none-any.whl (870 kB)
  Using cached rasterio-1.2.10.tar.gz (2.3 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [2 lines of output]
      INFO:root:Building on Windows requires extra options to setup.py to locate needed GDAL files. More information is available in the README.
      ERROR: A GDAL API version must be specified. Provide a path to gdal-config using a GDAL_CONFIG environment variable or use a GDAL_VERSION environment variable.
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [5]:
!pip install folium

In [6]:
!pip install mapboxgl

### unwanted

In [ ]:
for i in range(len(df_total)):
    try: 
        try:
            df_total.loc[i, 'available_month'] = df_total.loc[i, 'a'].month
        except: 
            df_total.loc[i, 'available_month'] = df.loc[i, 'a']
    except: print(df_total.loc[i, 'available_month'])

nan
nan
nan
nan
nan
nan
nan
nan


In [ ]:
df_total.available_month

0       1
1       5
2       1
3       1
4       1
       ..
2208    1
2209    1
2210    4
2211    5
2212    5
Name: available_month, Length: 2213, dtype: object

In [ ]:
len(re.findall('\d+', 'approx. 1400sq.ft. (very large!)')) == 1

True

In [ ]:
df_total.loc[:, 'square feet custom'] = df_total.loc[:, 'sq_feet']#.copy()

In [ ]:
df_total.loc[15, 'sq_feet']

0

In [ ]:
df_total[df_total.sq_feet==0]

,availability,a,city,community,latitude,longitude,link,type,price,beds,sq_feet,baths,cats,dogs,utility_heat,utility_electricity,utility_water,utility_cable,utility_internet,square feet custom
15,Immediate,2000-01-01,Calgary,Triwood,51.087566,-114.110119,/ab/calgary/rentals/apartment/1-bedroom/triwood/pet-friendly/359909,Apartment,845.0,studio,0,1,2,2,1,0,1,0,0,0
149,Immediate,2000-01-01,Calgary,Beltline,51.040137,-114.089793,/ab/calgary/rentals/apartment/1-bedroom/beltline/pet-friendly/355714,Apartment,1099.0,1,0,1,2,2,1,0,1,0,0,0
245,Immediate,2000-01-01,Calgary,Stanley Park,51.012017,-114.066371,/ab/calgary/rentals/apartment/1-bedroom/stanley-park/pet-friendly/433935,Apartment,1273.0,1,0,1,2,2,0,0,0,0,0,0
246,Immediate,2000-01-01,Calgary,Cliff Bungalow,51.038917,-114.075574,/ab/calgary/rentals/apartment/1-bedroom/cliff-bungalow/pet-friendly/433934,Apartment,1465.0,1,0,1,2,2,0,0,0,0,0,0
462,Immediate,2000-01-01,Calgary,Stanley Park,51.012017,-114.066371,/ab/calgary/rentals/apartment/1-bedroom/stanley-park/pet-friendly/433935,Apartment,1717.0,2,0,2,2,2,0,0,0,0,0,0
463,Immediate,2000-01-01,Calgary,Cliff Bungalow,51.038917,-114.075574,/ab/calgary/rentals/apartment/1-bedroom/cliff-bungalow/pet-friendly/433934,Apartment,1676.0,2,0,1,2,2,0,0,0,0,0,0


In [51]:
df_total.a

0       2000-01-01 00:00:00
1       2022-05-01 00:00:00
2       2000-01-01 00:00:00
3       2000-01-01 00:00:00
4       2000-01-01 00:00:00
               ...         
2208    2000-01-01 00:00:00
2209    2000-01-01 00:00:00
2210    2022-04-15 00:00:00
2211    2022-05-01 00:00:00
2212    2022-05-15 00:00:00
Name: a, Length: 2213, dtype: object

In [ ]:
len('+ / -  1000 sq feet')

19

In [ ]:
re.findall('[\w\.\!\- ]*\d+ *\+ *\d+ [\w\.\!\- ]*', '996 + 900 SQ FT BALCONY!')#[0]

['996 + 900 SQ FT BALCONY!']

In [ ]:
re.findall('\d+', '650  balcony')

['650']

In [ ]:
re.findall('[\w\.\!\- ]*\d+ *- *\d+[\w\.\!\- ]*', df_total.loc[1, 'sq_feet'])#[0]

[]

In [52]:
df.a

0       2000-01-01
1       2022-05-01
2       2000-01-01
3       2000-01-01
4       2000-01-01
           ...    
2028    3000-00-00
2029    3000-00-00
2030    3099-01-01
2031    3099-01-01
2032    3099-01-01
Name: a, Length: 2033, dtype: object

In [24]:
df_total.columns
checking_cols = [
    'availability', 'city', 'community', 'type', 'beds',
     'baths', 'cats', 'dogs'
     ]

In [25]:
for col in checking_cols:
    print('\ncolumn ', colored(col, 'green'))
    print(df_total.loc[:, col].unique())


column  availability
['Immediate' 'May 01' nan 'Apr 15' 'Apr 30' 'May 15' 'Apr 17' 'Apr 20'
 'Apr 21' 'Apr 28' 'May 06' 'May 07' 'May 09' 'May 10' 'May 13' 'May 20'
 'Jun 01' 'Jun 15' 'Jul 01' 'Jul 15' 'Aug 01' 'Sep 01' 'Negotiable'
 'Apr 07' 'Apr 13' 'Apr 14' 'Jun 30' 'Nov 01' 'Apr 11' 'Apr 09' 'Apr 12'
 'Apr 16' 'Apr 10' 'May 05' 'May 31' 'Jul 08' 'Oct 01' 'Apr 18' 'May 02'
 'May 16' 'Apr 08' 'Jun 05' 'Sep 24' 'Jun 20' 'Jun 22' 'Jun 02' 'Apr 25'
 'May 03' 'May 18' 'May 21' 'May 28' 'Jun 07' 'Jun 16']

column  city
['Calgary' nan]

column  community
['Cliff Bungalow' 'Thorncliffe' 'Forest Heights' 'Lower Mount Royal'
 'Beltline' 'Crescent Heights' 'Braeside' 'Mission' 'Bridgeland' 'Sunalta'
 'Dover' 'Haysboro' 'Downtown' 'Triwood' 'Connaught' 'Bankview' 'Altadore'
 'Scarboro' 'Brentwood' 'Huntington Hills' 'Renfrew' 'South Calgary'
 'Greenview' 'Woodlands' 'Shaganappi' 'Windsor Park' 'Forest Lawn'
 'Killarney' 'Bowness' 'Mount Royal' 'Inglewood' 'Deer Run' 'Dalhousie'
 'Willow Park' 

In [40]:
np.floor(3.5)

3.0

In [39]:
df_total[df_total.baths.isna()]

,availability,a,city,community,latitude,longitude,link,type,price,beds,sq_feet,baths,cats,dogs,utility_heat,utility_electricity,utility_water,utility_cable,utility_internet


In [29]:
int(re.findall('\d', '3+den')[0])

3

In [32]:
df_total.reset_index(inplace=True, drop=True)

In [27]:
df_total[df_total.beds=='studio']


,availability,a,city,community,latitude,longitude,link,type,price,beds,sq_feet,baths,cats,dogs,utility_heat,utility_electricity,utility_water,utility_cable,utility_internet
0,Immediate,2000-01-01,Calgary,Cliff Bungalow,51.036696,-114.074492,/ab/calgary/rentals/apartment/studio/cliff-bungalow/357976,Apartment,525.0,studio,420,1,0,0,1,1,1,0,0
4,Immediate,2000-01-01,Calgary,Beltline,51.041265,-114.066508,/ab/calgary/rentals/apartment/studio/beltline/pet-friendly/339295,Apartment,695.0,studio,205,1,2,2,0,0,0,0,0
5,Immediate,2000-01-01,Calgary,Crescent Heights,51.064555,-114.064615,/ab/calgary/rentals/apartment/studio/crescent-heights/3898,Apartment,895.0,studio,Approx 345,1,0,0,1,0,1,0,0
9,Immediate,2000-01-01,Calgary,Mission,51.036992,-114.075232,/ab/calgary/rentals/apartment/1-bedroom/mission/pet-friendly/466381,Apartment,875.0,studio,350,1,2,2,1,0,1,0,0
14,Immediate,2000-01-01,Calgary,Haysboro,50.977276,-114.076662,/ab/calgary/rentals/apartment/1-bedroom/haysboro/pet-friendly/75289,Apartment,950.0,studio,374,1,2,2,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
947,NaN,3099-01-01,Calgary,Scarboro,51.040496,-114.111504,/ab/calgary/rentals/basement/studio/scarboro/481472,Basement,900.0,studio,350,1,0,0,1,1,1,0,1
952,May 01,2022-05-01,Calgary,Banff Trail,51.067351,-114.116672,/ab/calgary/rentals/condo/studio/banff-trail/pet-friendly/484052,Condo,1000.0,studio,255,1,2,2,1,1,1,0,1
1039,Apr 16,2022-04-16,Calgary,Connaught,51.039142,-114.080065,/ab/calgary/rentals/condo/studio/connaught/80161,Condo,1175.0,studio,400,1,0,0,1,0,1,0,0
1235,Apr 15,2022-04-15,Calgary,Beltline,51.038769,-114.079079,/ab/calgary/rentals/condo/studio/beltline/pet-friendly/295734,Condo,1600.0,studio,450,1,2,2,1,1,1,0,1


In [ ]:
len(df[(~df.price2.isna())&(~df.sq_feet2.isna())&((~df.beds.isna()))&(~df.baths2.isna())])

180

In [43]:
df[['price', 'price2']]

,price,price2
0,999.0,NaN
1,974.0,999.0
2,899.0,950.0
3,849.0,999.0
4,800.0,1000.0
...,...,...
182,1245.0,2300.0
183,2700.0,2800.0
184,1567.0,1839.0
185,2925.0,3650.0


In [27]:
df[df.availability != 'Immediate'].availability.unique()
#df.loc[:, 'availability_month'] 

array(['May 01', 'Apr 15', 'Apr 30', 'call for availability', 'May 15',
       'Apr 07', 'Apr 17', 'Apr 20', 'Apr 21', 'Apr 28', 'May 06',
       'May 07', 'May 09', 'May 10', 'May 13', 'May 20', 'Jun 01',
       'Jun 15', 'Jul 01', 'Jul 15', 'Aug 01', 'Sep 01', 'Apr 13',
       'Apr 14', 'Jun 30', 'Nov 01', 'Apr 11', 'Apr 09', 'Apr 10',
       'Apr 12', 'Apr 16', 'May 26', 'Apr 08', 'May 05', 'May 31',
       'Jul 08', 'Oct 01', 'Apr 18', 'May 02', 'May 16', 'Jun 05',
       'Sep 24', 'Jun 20', 'Jun 22', 'Jun 02', 'Apr 25', 'May 03',
       'May 18', 'May 21', 'May 28', 'Jun 07', 'Jun 16'], dtype=object)